# Firehose

Here's @skyware/jetstream version

In [ ]:
//| export

import { Jetstream } from "@skyware/jetstream";
import { ids } from "tinychat/api/lexicons.ts";
import { z } from "zod";
import { Database } from "tinychat/db.ts";
import { getProfile } from "tinychat/bsky.ts";

const atURI = (did: string, collection: string, rkey: string) =>
  `at://${did}/${collection}/${rkey}`;

const baseCommitSchema = z.object({
  rev: z.string(),
  operation: z.string(),
  collection: z.string(),
  rkey: z.string(),
  cid: z.string(),
});

// Change to object schema that can be extended
const makeBaseSchema = <T extends z.ZodTypeAny>(recordSchema: T) =>
  z
    .object({
      did: z.string(),
      time_us: z.number(),
      commit: baseCommitSchema.extend({
        record: recordSchema,
      }),
    })
    .transform((d) => {
      if (!d.commit) {
        throw new Error(`Invalid record: ${JSON.stringify(d, null, 2)}`);
      }
      return Object.assign({}, d, {
        uri: atURI(d.did, d.commit.collection, d.commit.rkey),
      });
    });

const newServerRecordSchema = makeBaseSchema(
  z.object({
    $type: z.literal(ids.ChatTinychatCoreServer),
    name: z.string(),
    channels: z.array(z.object({
      id: z.string(),
      name: z.string(),
    })).min(1),
  }),
);

const newMembershipRecordSchema = makeBaseSchema(
  z.object({
    $type: z.literal(ids.ChatTinychatCoreMembership),
    createdAt: z.string(),
    server: z.string(),
  }),
);

const newMessageRecordSchema = makeBaseSchema(
  z.object({
    $type: z.literal(ids.ChatTinychatCoreMessage),
    channel: z.string(),
    createdAt: z.string(),
    server: z.string(),
    text: z.string(),
  }),
);

export type NewServerRecord = z.infer<typeof newServerRecordSchema>;
export type NewMembershipRecord = z.infer<typeof newMembershipRecordSchema>;
export type NewMessageRecord = z.infer<typeof newMessageRecordSchema>;

const jetstream = new Jetstream({
  wantedCollections: Deno.env.get("JETSTREAM_WANTED_COLLECTIONS")!.split(","),
  endpoint: Deno.env.get("JETSTREAM_URL")!,
  // cursor: Number(cursorFile),
});

type JetstreamCleanup = () => void;

type JetstreamConfig = {
  db: Database;
  onNewServer: (m: NewServerRecord) => void;
  onNewMembership: (m: NewMembershipRecord) => void;
  onNewMessage: (m: NewMessageRecord) => void;
};

export function startJetstream(
  { onNewServer, onNewMembership, onNewMessage, db }: JetstreamConfig,
): JetstreamCleanup {
  console.log("Starting jetstream");

  // let intervalID: NodeJS.Timeout;
  // const cursorFile = fs.readFileSync("cursor.txt", "utf8");
  // if (cursorFile) ctx.logger.info(`Initiate jetstream at cursor ${cursorFile}`);

  jetstream.on("error", (err) => console.error(err));
  // jetstream.on("close", () => clearInterval(intervalID));

  jetstream.on("open", () => {
    // intervalID = setInterval(() => {
    //   if (jetstream.cursor) {
    //     fs.writeFile("cursor.txt", jetstream.cursor.toString(), (err) => {
    //       if (err) console.log(err);
    //     });
    //   }
    // }, 60000);
  });

  const syncUser = async (did: string) => {
    const profile = await getProfile(did);
    db.prepare(
      `
        INSERT INTO users (did, handle, display_name, avatar, description) VALUES (
          :did, :handle, :displayName, :avatar, :description
        ) ON CONFLICT(did) DO UPDATE SET
          handle = COALESCE(:handle, handle),
          display_name = COALESCE(:displayName, display_name),
          avatar = COALESCE(:avatar, avatar),
          description = COALESCE(:description, description
        )`
    ).run({
      did,
      handle: profile.handle,
      displayName: profile.displayName,
      avatar: profile.avatar,
      description: profile.description,
    });
  }


  // handle server updates
  jetstream.on(ids.ChatTinychatCoreServer, async (event) => {
    // we only do creates for now
    if (event.commit.operation !== "create") {
      return;
    }
    await syncUser(event.did);
    onNewServer(newServerRecordSchema.parse(event));
  });

  // handle membership updates
  jetstream.on(ids.ChatTinychatCoreMembership, async (event) => {
    // we only do creates for now
    if (event.commit.operation !== "create") {
      return;
    }
    await syncUser(event.did);
    onNewMembership(newMembershipRecordSchema.parse(event));
  });

  // handle new message
  jetstream.on(ids.ChatTinychatCoreMessage, async (event) => {
    // we only do creates for now
    if (event.commit.operation !== "create") {
      return;
    }
    console.log("New message >>>>>>>>>>>>>>>>>>>>", event);
    await syncUser(event.did);
    onNewMessage(newMessageRecordSchema.parse(event));
  });

  jetstream.start();
  console.log("Jetstream started");

  return () => {
    console.log("Stopping jetstream");
    jetstream.removeAllListeners();
    jetstream.close();
    console.log("Jetstream stopped");
  };
}

In [ ]:
import { assert } from "asserts";
import { TID } from "@atproto/common";
import { TinychatAgent } from "tinychat/agent.ts";
import { sleep } from "tinychat/utils.ts";
import { getDatabase } from "tinychat/db.ts";

Deno.test("jetstream", async (t) => {
  const servers: NewServerRecord[] = [];
  const memberships: NewMembershipRecord[] = [];
  const messages: NewMessageRecord[] = [];
  const cleanup = startJetstream({
    db: getDatabase(),
    onNewServer: (m: NewServerRecord) => {
      servers.push(m);
    },
    onNewMembership: (m: NewMembershipRecord) => {
      memberships.push(m);
    },
    onNewMessage: (m: NewMessageRecord) => {
      messages.push(m);
    },
  });
  const serverName = `test-${TID.nextStr()}`;
  const channelId = TID.nextStr();
  const agent = await TinychatAgent.create();
  const repo = await agent.agent.assertDid;

  // let's create a new chat server and watch it propagate through the system

  const chatServer = await agent.chat.tinychat.core.server.create({ repo }, {
    name: serverName,
    channels: [
      { id: channelId, name: "test channel" },
    ],
  });
  await agent.chat.tinychat.core.membership.create({ repo }, {
    server: chatServer.uri,
    createdAt: new Date().toISOString(),
  });
  await agent.chat.tinychat.core.message.create({
    repo,
  }, {
    channel: channelId,
    server: chatServer.uri,
    text: "Hello world",
    createdAt: new Date().toISOString(),
  });

  await sleep(2000);

  await t.step("spot check jetstream updates", () => {
    assert(servers.length > 0);
    assert(servers.some((s) => s.commit.record.name === serverName));
    assert(servers.some((s) => s.commit.record.channels.some((c) => c.id === channelId)));
    assert(memberships.length > 0);
    assert(memberships.some((m) => m.commit.record.server === chatServer.uri));
    assert(messages.length > 0);
    assert(
      messages.some((m) =>
        m.commit.record.text === "Hello world" &&
        m.commit.record.channel === channelId
      ),
    );
  });

  cleanup();
  await sleep(2000);
});